In [82]:
from PyPDF2 import PdfReader
import pandas as pd
import re
import numpy as np

pdf_path = "sem09.pdf"
page = PdfReader(pdf_path).pages[56]

df = pd.DataFrame(columns=['Entidad_Federativa', 'Depresion_Sem', 'Depresion_M', 'Depresion_F', 'Parkinson_Sem', 'Parkinson_M', 'Parkinson_F', 'Alzheimer_Sem', 'Alzheimer_M', 'Parkinson_F'])

lines = map(lambda x: re.split(r'\s{2,}', x), page.extract_text().split('\n'))

entidades = [
    "Aguascalientes", "Baja California", "Baja California Sur", "Campeche", 
    "Coahuila", "Colima", "Chiapas", "Chihuahua", "Distrito Federal", 
    "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "México", 
    "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", 
    "Querétaro", "Quintana Roo", "San Luis Potosí", "Sinaloa", "Sonora", 
    "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas", "TOTAL"
]

def filter_by_entity(line):
    return " ".join(line[0].split(' ')[:-1]) in entidades


filtered_lines = list(filter(filter_by_entity, lines))
filtered_lines        

[['Baja California 51', '72', '248', '11', '21', '25 -', '4', '15'],
 ['Baja California Sur 4', '11', '37 -', '4', '4 - - -'],
 ['Campeche 17', '24', '76', '1', '1', '6 -', '4', '3'],
 ['Coahuila 44', '51', '163', '5', '26', '23 -', '3', '6'],
 ['Colima 12', '27', '90', '3', '7', '11 - -', '2'],
 ['Chiapas 15', '17', '95', '3', '10', '5', '2', '4', '4'],
 ['Chihuahua 67', '65', '293', '8', '23', '24', '1', '1', '11'],
 ['Distrito Federal 309', '382', '1 047', '10', '24', '71', '1', '5', '5'],
 ['Durango 9', '26', '79 -', '2', '2 -', '2', '3'],
 ['Guanajuato 18', '6', '40', '2', '3', '5 - -', '1'],
 ['Guerrero 14', '21', '70', '10', '19', '8 -', '2', '4'],
 ['Hidalgo 23', '28', '98', '1', '12', '14 - - -'],
 ['Jalisco 49', '48', '190 -', '3', '5 -', '1', '3'],
 ['México 108', '154', '595', '12', '61', '47', '2', '12', '14'],
 ['Michoacán 39', '57', '210', '9', '21', '19', '2', '7', '8'],
 ['Morelos 65', '56', '220', '2', '19', '5', '1 -', '5'],
 ['Nayarit 10', '36', '136', '2', '7', '6'

In [83]:
def parse_first_item(item):
    """
    Separa el nombre del estado (parte de texto) de cualquier número que
    esté pegado al final.
    """
    # Reemplaza múltiples espacios por uno solo (por seguridad)
    item = re.sub(r'\s+', ' ', item).strip()
    
    # Busca una parte sin dígitos seguida opcionalmente de algo numérico
    match = re.match(r'([^\d]+)(.*)', item)
    if match:
        state_part = match.group(1).strip()  # texto (nombre estado)
        number_part = match.group(2).strip() # lo que quede (puede ser dígitos u otro)
    else:
        # Si no macha, consideramos todo como nombre
        state_part = item
        number_part = ""
    return state_part, number_part

def split_and_clean(items):
    """
    - Si el string contiene guiones, lo parte por espacios ('25 -' -> ['25','-']).
    - Si NO contiene guiones ('1 047'), elimina espacios internos para leerlo como un solo número ('1047').
    - Convierte dígitos a int, guiones a NaN.
    """
    output = []
    for piece in items:
        # Caso 1: Si hay un guion, partimos por espacio
        if '-' in piece:
            tokens = piece.split()
            for t in tokens:
                t = t.strip().replace(' ', '')  # elimina espacios internos
                if t.isdigit():
                    output.append(int(t))
                elif t == '-':
                    output.append(np.nan)
                else:
                    output.append(np.nan)
        else:
            # Caso 2: No hay guion => eliminar todos los espacios internos y tratarlo como un único valor
            piece_no_spaces = piece.replace(' ', '')
            if piece_no_spaces.isdigit():
                output.append(int(piece_no_spaces))
            else:
                # Si no es dígito limpio, lo convertimos a NaN
                output.append(np.nan)
    return output

cleaned_data = []
for row in filtered_lines:
    # 1) Separar el primer item (nombre estado + posible número)
    state, possible_num = parse_first_item(row[0])

    # 2) Construir lista con "possible_num" (si lo hay) y el resto de la fila
    #    a partir del segundo elemento.
    rest = []
    if possible_num:
        rest.append(possible_num)
    rest.extend(row[1:])  # Agregar el resto

    # 3) Dividir cada sub-elemento y limpiar
    numeric_values = split_and_clean(rest)

    # -- En este paso numeric_values puede tener más o menos de 9 elementos. --
    # Si quieres forzar a que cada fila tenga EXACTAMENTE 9 valores:
    if len(numeric_values) < 9:
        # Rellenar con NaN si faltan
        numeric_values += [np.nan]*(9 - len(numeric_values))
    elif len(numeric_values) > 9:
        # Recortar si sobran
        numeric_values = numeric_values[:9]

    # 4) Agregar la fila completa: [Estado] + 9 valores
    cleaned_data.append([state] + numeric_values)

# Definir columnas
columns = [
    'Entidad_Federativa',
    'Depresion_Sem','Depresion_M','Depresion_F',
    'Parkinson_Sem','Parkinson_M','Parkinson_F',
    'Alzheimer_Sem','Alzheimer_M','Alzheimer_F'
]

# Crear DataFrame
df = pd.DataFrame(cleaned_data, columns=columns)

# FORMATEAR NÚMEROS MAYORES DE 999
# (aunque con estos datos quizá no sea muy común)
def formato_miles(x):
    if pd.isna(x):
        return np.nan
    # Si es entero y > 999, poner formato "1,234"
    if isinstance(x, int) and x > 999:
        return f"{x:,}".replace(",", " ")
    return x

df.iloc[:, 1:] = df.iloc[:, 1:].applymap(formato_miles)

df

,Entidad_Federativa,Depresion_Sem,Depresion_M,Depresion_F,Parkinson_Sem,Parkinson_M,Parkinson_F,Alzheimer_Sem,Alzheimer_M,Alzheimer_F
0,Baja California,51,72,248,11.0,21.0,25.0,NaN,4.0,15.0
1,Baja California Sur,4,11,37,NaN,4.0,4.0,NaN,NaN,NaN
2,Campeche,17,24,76,1.0,1.0,6.0,NaN,4.0,3.0
3,Coahuila,44,51,163,5.0,26.0,23.0,NaN,3.0,6.0
4,Colima,12,27,90,3.0,7.0,11.0,NaN,NaN,2.0
5,Chiapas,15,17,95,3.0,10.0,5.0,2.0,4.0,4.0
6,Chihuahua,67,65,293,8.0,23.0,24.0,1.0,1.0,11.0
7,Distrito Federal,309,382,1 047,10.0,24.0,71.0,1.0,5.0,5.0
8,Durango,9,26,79,NaN,2.0,2.0,NaN,2.0,3.0
9,Guanajuato,18,6,40,2.0,3.0,5.0,NaN,NaN,1.0
